# **1. Mount Drive + Install Libraries**

In [1]:
# ============================================
# 📌 CELL 1 — Mount Drive + Install Libraries
# ============================================

from google.colab import drive
drive.mount('/content/drive')

# Install required packages
!pip install tldextract python-whois bs4 nltk textblob textstat spacy scikit-learn --quiet
!python -m spacy download en_core_web_sm --quiet

# NLTK data downloads
import nltk
nltk.download("punkt")
nltk.download("punkt_tab")
nltk.download("stopwords")
nltk.download("wordnet")

print("Drive mounted & all libraries installed successfully!")

Mounted at /content/drive
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.4/107.4 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.4/176.4 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 66.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 85.0 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


Drive mounted & all libraries installed successfully!


# **2.Import Dependencies**

In [2]:
# ============================================
# 📌 CELL 2 — Import Dependencies
# ============================================

import pandas as pd
import numpy as np
import requests, re, random, tldextract, whois
from bs4 import BeautifulSoup

# NLP
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from textblob import TextBlob
import textstat
import spacy

# ML utilities
from sklearn.feature_extraction.text import TfidfVectorizer

# Load spaCy model
nlp = spacy.load("en_core_web_sm")

# Setup NLP components
stop_words = set(stopwords.words("english"))
lemm = WordNetLemmatizer()

print("All dependencies imported successfully!")

All dependencies imported successfully!


# **3.FULL FEATURE EXTRACTION PIPELINE**

In [3]:
# ===================================================================
# 📌 CELL 3 — PHASE 1: FULL FEATURE EXTRACTION PIPELINE (Single Cell)
# ===================================================================

# ---------------------------
# 1. TEXT PREPROCESSING
# ---------------------------
def preprocess_text(text):
    if not isinstance(text, str) or text.strip() == "":
        return "no_content"

    text = re.sub(r"<.*?>", " ", text)
    text = re.sub(r"http\S+", " ", text)
    text = re.sub(r"[^A-Za-z\s]", " ", text)
    text = text.lower()

    tokens = nltk.word_tokenize(text)
    tokens = [lemm.lemmatize(t) for t in tokens if t not in stop_words]

    cleaned = " ".join(tokens)
    return cleaned if cleaned.strip() else "no_content"


# ---------------------------
# 2. URL FEATURES
# ---------------------------
def extract_url_features(url):
    info = {}
    info["url_length"] = len(url)
    info["num_digits"] = sum(c.isdigit() for c in url)
    info["num_special_chars"] = len(re.findall(r"[\W_]", url))
    info["has_https"] = 1 if url.startswith("https") else 0

    d = tldextract.extract(url)
    info["domain"] = d.domain
    info["suffix"] = d.suffix

    return info


# ---------------------------
# 3. DOMAIN AGE (WHOIS)
# ---------------------------
def extract_domain_age(url):
    try:
        data = whois.whois(url)
        c = data.creation_date

        if isinstance(c, list):
            c = c[0]

        age = (pd.Timestamp.now() - pd.Timestamp(c)).days
        return age if age > 0 else -1

    except:
        return -1


# ---------------------------
# 4. SYNTHETIC DATASET (100 REAL + 100 FAKE WEBSITES)
# ---------------------------

# Real websites (label=0)
real_websites = [
    "https://amazon.in", "https://flipkart.com", "https://ajio.com",
    "https://myntra.com", "https://nike.com", "https://apple.com",
    "https://hp.com", "https://dell.com", "https://samsung.com",
    "https://bbc.com", "https://nytimes.com"
] * 10
real_websites = real_websites[:100]

real_placeholder_text = (
    "This is a legitimate website with proper product information, "
    "company details, customer service info, and policies."
)
real_texts = [real_placeholder_text] * 100

# Fake websites (label=1)
fake_patterns = [
    "discount70", "mega-sale", "flash-offer", "exclusive-deal",
    "promo", "cheap-price", "wallet-verify", "bonus-offer"
]
tlds = ["xyz", "shop", "top", "online", "buzz", "store"]

fake_websites = []
for i in range(100):
    brand = random.choice(["amazon", "flipkart", "ajio", "myntra", "nike", "meesho"])
    pattern = random.choice(fake_patterns)
    tld = random.choice(tlds)
    fake_websites.append(f"https://{brand}-{pattern}.{tld}")

# Fake HTML pages with scam keywords
fake_html_template = """
<h1>{headline}</h1>
<p>{line1}</p>
<p>{line2}</p>
<p>{line3}</p>
"""
scam_sentences = [
    ("LIMITED OFFER! 90% DISCOUNT!", "Buy now!", "Exclusive deal!", "Hurry, ending soon!"),
    ("FLASH SALE! EVERYTHING MUST GO!", "Lowest price!", "No returns.", "Order today!"),
    ("MEGA DEAL! 80% OFF!", "Free shipping!", "COD available!", "Best price guaranteed!")
]
fake_texts = []
for _ in range(100):
    h, l1, l2, l3 = random.choice(scam_sentences)
    fake_texts.append(fake_html_template.format(headline=h, line1=l1, line2=l2, line3=l3))


# ---------------------------
# 5. MERGE INTO ONE DATAFRAME
# ---------------------------
df_real = pd.DataFrame({"url": real_websites, "raw_text": real_texts, "label": 0})
df_fake = pd.DataFrame({"url": fake_websites, "raw_text": fake_texts, "label": 1})
df = pd.concat([df_real, df_fake], ignore_index=True)


# ---------------------------
# 6. NLP FEATURE EXTRACTION
# ---------------------------
SCAM_KEYWORDS = [
    "discount", "free", "limited", "offer", "guarantee", "sale",
    "buy now", "cash on delivery", "clearance", "exclusive", "hurry"
]

def scam_keyword_count(text):
    return sum(1 for kw in SCAM_KEYWORDS if kw in text.lower())

def sentiment_score(text):
    return TextBlob(text).sentiment.polarity

def readability_score(text):
    try:
        return textstat.flesch_reading_ease(text)
    except:
        return 0

def entity_count(text):
    try:
        doc = nlp(text)
        return len(doc.ents)
    except:
        return 0


# Apply to all rows
records = []

for _, row in df.iterrows():
    url = row["url"]
    raw = row["raw_text"]
    clean = preprocess_text(raw)

    feats = extract_url_features(url)
    feats["domain_age_days"] = extract_domain_age(url)

    feats.update({
        "raw_text": raw,
        "clean_text": clean,
        "text_length": len(raw),
        "scam_keyword_score": scam_keyword_count(clean),
        "sentiment": sentiment_score(clean),
        "readability": readability_score(clean),
        "entity_count": entity_count(clean),
        "label": row["label"]
    })

    records.append(feats)

df_final = pd.DataFrame(records)


# ---------------------------
# 7. TF-IDF VECTORIZATION
# ---------------------------
tfidf = TfidfVectorizer(max_features=3000)
df_final["clean_text"] = df_final["clean_text"].replace("", "no_content")
tfidf_matrix = tfidf.fit_transform(df_final["clean_text"])

print("🎉 PHASE 1 COMPLETED! Your dataset is ready.")
df_final.head(10)

ERROR:whois.whois:Error trying to connect to socket: closing socket - [Errno 104] Connection reset by peer
ERROR:whois.whois:Error trying to connect to socket: closing socket - [Errno 104] Connection reset by peer
ERROR:whois.whois:Error trying to connect to socket: closing socket - [Errno 104] Connection reset by peer
ERROR:whois.whois:Error trying to connect to socket: closing socket - [Errno 104] Connection reset by peer
ERROR:whois.whois:Error trying to connect to socket: closing socket - [Errno 104] Connection reset by peer
ERROR:whois.whois:Error trying to connect to socket: closing socket - [Errno 104] Connection reset by peer
ERROR:whois.whois:Error trying to connect to socket: closing socket - [Errno 104] Connection reset by peer


🎉 PHASE 1 COMPLETED! Your dataset is ready.


,url_length,num_digits,num_special_chars,has_https,domain,suffix,domain_age_days,raw_text,clean_text,text_length,scam_keyword_score,sentiment,readability,entity_count,label
0,17,0,4,1,amazon,in,-1,This is a legitimate website with proper produ...,legitimate website proper product information ...,115,0,0.0,-27.366364,0,0
1,20,0,4,1,flipkart,com,-1,This is a legitimate website with proper produ...,legitimate website proper product information ...,115,0,0.0,-27.366364,0,0
2,16,0,4,1,ajio,com,-1,This is a legitimate website with proper produ...,legitimate website proper product information ...,115,0,0.0,-27.366364,0,0
3,18,0,4,1,myntra,com,-1,This is a legitimate website with proper produ...,legitimate website proper product information ...,115,0,0.0,-27.366364,0,0
4,16,0,4,1,nike,com,-1,This is a legitimate website with proper produ...,legitimate website proper product information ...,115,0,0.0,-27.366364,0,0
5,17,0,4,1,apple,com,-1,This is a legitimate website with proper produ...,legitimate website proper product information ...,115,0,0.0,-27.366364,0,0
6,14,0,4,1,hp,com,-1,This is a legitimate website with proper produ...,legitimate website proper product information ...,115,0,0.0,-27.366364,0,0
7,16,0,4,1,dell,com,-1,This is a legitimate website with proper produ...,legitimate website proper product information ...,115,0,0.0,-27.366364,0,0
8,19,0,4,1,samsung,com,-1,This is a legitimate website with proper produ...,legitimate website proper product information ...,115,0,0.0,-27.366364,0,0
9,15,0,4,1,bbc,com,-1,This is a legitimate website with proper produ...,legitimate website proper product information ...,115,0,0.0,-27.366364,0,0


# **4.SAVE PHASE 1 OUTPUTS**

In [4]:
# ============================================================
# 📌 SAVE PHASE 1 OUTPUTS (df_final, tfidf, tfidf_matrix)
# ============================================================

import joblib
import os

# Create folders if they don’t exist
base_dir = "/content/drive/MyDrive/scamscan_project"
os.makedirs(f"{base_dir}/models", exist_ok=True)
os.makedirs(f"{base_dir}/outputs", exist_ok=True)

# 1. Save df_final dataset
df_final_path = f"{base_dir}/outputs/df_final.csv"
df_final.to_csv(df_final_path, index=False)

# 2. Save TF-IDF vectorizer
tfidf_path = f"{base_dir}/models/tfidf_vectorizer.pkl"
joblib.dump(tfidf, tfidf_path)

# 3. Save TF-IDF matrix (sparse)
tfidf_matrix_path = f"{base_dir}/models/tfidf_matrix.pkl"
joblib.dump(tfidf_matrix, tfidf_matrix_path)

print("🎉 Phase 1 outputs saved successfully!")
print("📁 Dataset saved to:", df_final_path)
print("📁 TF-IDF vectorizer saved to:", tfidf_path)
print("📁 TF-IDF matrix saved to:", tfidf_matrix_path)

🎉 Phase 1 outputs saved successfully!
📁 Dataset saved to: /content/drive/MyDrive/scamscan_project/outputs/df_final.csv
📁 TF-IDF vectorizer saved to: /content/drive/MyDrive/scamscan_project/models/tfidf_vectorizer.pkl
📁 TF-IDF matrix saved to: /content/drive/MyDrive/scamscan_project/models/tfidf_matrix.pkl


In [5]:
list(df_final.columns)

['url_length',
 'num_digits',
 'num_special_chars',
 'has_https',
 'domain',
 'suffix',
 'domain_age_days',
 'raw_text',
 'clean_text',
 'text_length',
 'scam_keyword_score',
 'sentiment',
 'readability',
 'entity_count',
 'label']